In [13]:

import os
print("Current working directory:", os.getcwd())
import importlib

import sys
sys.path.append('/data')
print(os.listdir('/data'))
print(os.getcwd())

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
#from model.init import model
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from model.Multiagent import MultiAgentFusion
# GNNs (Graph Neural Network) 
from model.mlp import MLPModel
from sklearn.model_selection import KFold
import torch.optim as optim
# MAMBA 
import torch
import torch.nn as nn
#from model.mamba import Mamba
# LSTM 
from model.lstm import LSTM
# GRU 
from model.gru import GRUModel
# Neural Network 
from model.neural import NeuralODE
# Transformer 
from model.transformer import TransformerModel
import numpy as np
import shap
from mamba_ssm import Mamba
import os
from sklearn.preprocessing import LabelEncoder
#os.environ['CUDA_VISIBLE_DEVICES'] = "0"
print(torch.__version__)  # 查看torch当前版本号
print(torch.version.cuda)  # 编译当前版本的torch使用的cuda版本号
print(torch.cuda.is_available())  # 查看当前cuda是否可用于当前版本的Torch


Current working directory: /
['gzh', 'run.ipynb', 'Untitled-1.py', 'bert-base-uncased', 'All(1).xlsx', 'data.csv', 'run_.py', 'Untitled-2.py', 'model', 'run.py', 'transformer.py', '__pycache__', 'run_weida.ipynb', 'TrafficLabelling']
/
1.13.0+cu117
11.7
True


In [14]:
data = pd.read_csv('/data/TrafficLabelling/end.csv').values[0:10000]
le = LabelEncoder()
le.fit(data[:,-1])
data[:,-1]= le.transform(data[:,-1])
X_train, X_test, y_train, y_test = train_test_split(data[:,:-1].astype('float64'), data[:,-1].astype('float64'), test_size=0.2)
X_train=X_train[:, :5] 
X_test=X_test[:, :5] 
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
num_classes = len(np.unique(y_train))


In [9]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [15]:
from transformers import BertModel


class Agent:
    def __init__(self, model, optimizer, loss_fn, device):
        self.model = model.to(device)  # 把模型移动到指定设备
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.reward = 0
        self.device = device
       
    def train(self, X, y):
        self.model.train()
        inputs = torch.tensor(X, dtype=torch.float32).to(self.device)  # 将输入数据放到指定设备
        targets = torch.tensor(y, dtype=torch.long).to(self.device)  # 将标签放到设备
        self.optimizer.zero_grad()
        outputs = self.model(inputs)
        loss = self.loss_fn(outputs, targets)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def predict(self, X):
        self.model.eval()
        inputs = torch.tensor(X, dtype=torch.float32, requires_grad=True).to(self.device)
        outputs = self.model(inputs)
        return outputs

    def update_reward(self, reward):
        self.reward += reward

    def adjust_learning_rate(self):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = max(0.001, 0.01 + self.reward * 0.001)

model = MLPModel(input_size=5, hidden_size=64, output_size=num_classes)
mlp_model = MLPModel(input_size=5, hidden_size=64, output_size=num_classes)
gru_model = GRUModel(input_size=5, hidden_layer_size=64, num_layers=2, output_size=num_classes)
lstm_model = LSTM(input_size=5, hidden_layer_size=64, num_layers=2, output_size=num_classes)
batch, length, dim = 2, 64, 768
mamba_model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor # 64
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
    use_fast_path=False,
).to("cuda")
neural_model = NeuralODE()
transformer_model = TransformerModel(output_size=num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 创建每个代理时传入设备
agents = [
    Agent(
        model=mlp_model,
        optimizer=optim.Adam(mlp_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=gru_model,
        optimizer=optim.Adam(gru_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=lstm_model,
        optimizer=optim.Adam(lstm_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=mamba_model,
        optimizer=optim.Adam(mamba_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=neural_model,
        optimizer=optim.Adam(neural_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=transformer_model,
        optimizer=optim.Adam(transformer_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
]

# Main loop for training, fusion, feedback, and optimization
num_iterations = 5
train_losses = []
val_losses = []
accuracy_scores = []
agent_outputs = []

In [16]:
# Training and updating loop
for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}")

    # Each agent trains on mini-batches
    agent_outputs = []
    for agent in agents:
        for epoch in range(10):  # Use smaller epochs to avoid long runtime
            loss = agent.train(train_loader)  # Use DataLoader
            print(f"Epoch {epoch + 1}, Loss: {loss}")
        agent_outputs.append(agent.predict(X_test))  # Still use X_test for predictions

    # Calculate FB distribution
    fb_distribution = np.mean(agent_outputs, axis=0)

    # Further training based on FB distribution feedback
    for i, agent in enumerate(agents):
        fb_loss = agent.loss_fn(
            torch.tensor(agent_outputs[i], dtype=torch.float32).to(device),
            torch.tensor(fb_distribution, dtype=torch.float32).to(device)
        )
        agent.optimizer.zero_grad()
        fb_loss.backward()
        agent.optimizer.step()

    # Shapley value calculation
    shapley_values = []
    for agent in agents:
        explainer = shap.KernelExplainer(agent.predict, X_train)  # X_train should be small enough for SHAP
        shap_values = explainer.shap_values(X_test)
        shapley_values.append(np.mean(shap_values))  # Aggregate SHAP values

    # Reward and learning rate adjustment
    for i, agent in enumerate(agents):
        reward = shapley_values[i]
        agent.update_reward(reward)
        agent.adjust_learning_rate()

    # Display rewards
    for i, agent in enumerate(agents):
        print(f"Agent {i + 1} reward after iteration {iteration + 1}: {agent.reward}")

Iteration 1


TypeError: train() missing 1 required positional argument: 'y'

In [11]:
for iteration in range(num_iterations):
    print(f"Iteration {iteration+1}")
    
    # 每个 agent 训练并生成预测分布
    agent_outputs = []
    for agent in agents:
        for epoch in range(100):
            loss = agent.train(X_train, y_train)
        agent_outputs.append(agent.predict(X_test))

    # 计算 FB 分布
    fb_distribution = np.mean(agent_outputs, axis=0)

    # 每个 agent 基于 FB 分布反馈进行进一步训练
    for i, agent in enumerate(agents):
        fb_loss = agent.loss_fn(torch.tensor(agent_outputs[i], dtype=torch.float32), 
                                torch.tensor(fb_distribution, dtype=torch.float32))
        agent.optimizer.zero_grad()
        fb_loss.backward()
        agent.optimizer.step()

    # 计算 Shapley 值
    shapley_values = []
    for agent in agents:
        explainer = shap.KernelExplainer(agent.predict, X_train)
        shap_values = explainer.shap_values(X_test)
        shapley_values.append(np.mean(shap_values))
    
    # 奖惩机制
    for i, agent in enumerate(agents):
        reward = shapley_values[i]
        agent.update_reward(reward)
        agent.adjust_learning_rate()

    # 显示每轮训练后的奖励
    for i, agent in enumerate(agents):
        print(f"Agent {i+1} reward after iteration {iteration+1}: {agent.reward}")

# 最终输出每个 agent 的累计奖励及其贡献度
for i, agent in enumerate(agents):
    print(f"Final Reward for Agent {i+1}: {agent.reward}")

Iteration 1


RuntimeError: size mismatch (got input: [1], target: [8000])